<a href="https://colab.research.google.com/github/dvillasxUdg/ti_proyecto_final/blob/main/Proyecto_Final_Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Module and Init values

In [2]:
pip install pybaselines

In [3]:
import json
import numpy as np
import panel as pn
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from pybaselines import Baseline, utils

import panel as pn
import panel.widgets as pnw
pn.extension('plotly')

with open('average_coherence_data.json', 'r') as json_file:
    loaded_coherence_data = json.load(json_file)

participants = [
    'ADJRR', 'CESARMDG', 'EMR', 'HMM', 'JAVI', 'JSDR', 'LESR', 'MEGA', 'VMAP',
    'ABN', 'AGM', 'AJJDT', 'AMCJ', 'AMGP', 'IGNR', 'MFGM', 'MJLS', 'NYHG', 'SDAMG'
]
WORD_6 = '6'
WORD_7 = '7'
BAND = 'delta'
CHANNELS = ['F3', 'F4', 'C3', 'C4', 'P3', 'P4', 'Fz', 'Cz', 'Pz']

# Functions

In [4]:
# Plot Coherences values
def plot_coherence(coherence_matrix):
    coherence_matrix = coherence_matrix[::-1]

    trace = go.Heatmap(z=coherence_matrix,
                       x=CHANNELS,
                       y=CHANNELS[::-1],
                       colorscale='Viridis', zmin=0, zmax=1)

    layout = go.Layout(
        xaxis=dict(showline=True, showgrid=False, tickvals=list(range(len(CHANNELS))), ticktext=CHANNELS),
        yaxis=dict(showline=True, showgrid=False, tickvals=list(range(len(CHANNELS))), ticktext=CHANNELS[::-1]),
        autosize=False,
        width=400,
        height=400,
    )

    fig = go.Figure(data=[trace], layout=layout)
    return fig

# Plot Step length
def plot_histo(x):
  fig = go.Figure()
  fig.add_trace(go.Histogram(
      x=x,
      xbins=dict(
        size=0.01
    )))
  return fig

# Update panels
def update_panels(event):
    # Widgets values
    metric = metric_selector.value
    participant_name = participants_selector.value

    # Coherence values
    coherence_6 = loaded_coherence_data[participant_name][WORD_6][BAND]
    coherence_7 = loaded_coherence_data[participant_name][WORD_7][BAND]

    if metric == 'Coherencia':
      fig_cohe_6 = plot_coherence(coherence_6)
      fig_cohe_7 = plot_coherence(coherence_7)
    else:
      cohe_values_6 = np.asarray(coherence_6).reshape(-1)
      cohe_values_7 = np.asarray(coherence_7).reshape(-1)
      fig_cohe_6 = plot_histo(cohe_values_6)
      fig_cohe_7 = plot_histo(cohe_values_7)


    fig_combined = make_subplots(rows=1, cols=2, subplot_titles=[f'Congruente palabra {WORD_6}', f'Incongruente palabra {WORD_7}'])
    for trace in fig_cohe_6.data:
      fig_combined.add_trace(trace, row=1, col=1)
    for trace in fig_cohe_7.data:
      fig_combined.add_trace(trace, row=1, col=2)


    if metric != 'Coherencia':
      fig_combined.update_layout(
        xaxis_title_text='Value', # xaxis label
        yaxis_title_text='Count', # yaxis label
        )

    panel.object = fig_combined



# Main

In [5]:
# Widgets
participants_selector = pn.widgets.Select(name='Participante', options=participants)
metric_selector = pn.widgets.Select(name='Metrica', options=['Coherencia', 'Step Length'])

# Watcher
participants_selector.param.watch(update_panels, 'value')
metric_selector.param.watch(update_panels, 'value')

# Init values
participant_name = participants_selector.value
coherence_6 = loaded_coherence_data[participant_name][WORD_6][BAND]
coherence_7 = loaded_coherence_data[participant_name][WORD_7][BAND]

initial_fig_combined = make_subplots(rows=1, cols=2, subplot_titles=[f'Congruente palabra {WORD_6}', f'Incongruente palabra {WORD_7}'])

for fig, col in [(plot_coherence(coherence_6), 1),
                 (plot_coherence(coherence_7), 2)]:
    for trace in fig.data:
        initial_fig_combined.add_trace(trace, row=1, col=col)

panel = pn.pane.Plotly(initial_fig_combined)

# Set dashboard
layout = pn.Row(
    pn.Column(
        participants_selector,
        metric_selector
    ),
    pn.Column(
        panel,
        width=1000
    )
)

layout.servable()


Traceback (most recent call last):
 File "/usr/local/lib/python3.10/dist-packages/pyviz_comms/__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "/usr/local/lib/python3.10/dist-packages/panel/viewable.py", line 472, in _on_msg
 doc.unhold()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/document.py", line 776, in unhold
 self.callbacks.unhold()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 431, in unhold
 self.trigger_on_change(event)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 408, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 443, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/util/callback_manager.py", line 185, in invoke
 callback(attr, old, new)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 469, in _comm_change
 state._handle_exception(e)
 File "/usr/local/lib/python3.10/dist-packages/panel/io/state.py", line 436, in _handle_exception
 raise exception
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 467, in _comm_change
 self._schedule_change(doc, comm)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 449, in _schedule_change
 self._change_event(doc)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 445, in _change_event
 self._process_events(events)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 384, in _process_events
 self.param.update(**self_events)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 1902, in update
 self_._batch_call_watchers()
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2063, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2025, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 854, in _replace_pane
 new_object = self.eval(self.object)
 File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 813, in eval
 return eval_function(function)
 File "/usr/local/lib/python3.10/dist-packages/panel/util/__init__.py", line 325, in eval_function
 return function(*args, **kwargs)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 407, in _depends
 return func(*args, **kw)
 File "<ipython-input-9-5bd61d344502>", line 35, in data_plot
 matrix_avg = make_average(np.asanyarray(origin_data[participants_selector]['6']))
KeyError: 'JSDR'
 


 Traceback (most recent call last):
 File "/usr/local/lib/python3.10/dist-packages/pyviz_comms/__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "/usr/local/lib/python3.10/dist-packages/panel/viewable.py", line 472, in _on_msg
 doc.unhold()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/document.py", line 776, in unhold
 self.callbacks.unhold()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 431, in unhold
 self.trigger_on_change(event)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 408, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 443, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/util/callback_manager.py", line 185, in invoke
 callback(attr, old, new)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 469, in _comm_change
 state._handle_exception(e)
 File "/usr/local/lib/python3.10/dist-packages/panel/io/state.py", line 436, in _handle_exception
 raise exception
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 467, in _comm_change
 self._schedule_change(doc, comm)
 File "/usr/local/l

Row
    [0] Column
        [0] Select(name='Participante', options=['ADJRR', 'CESARMDG', ...], value='ADJRR')
        [1] Select(name='Metrica', options=['Coherencia', ...], value='Coherencia')
    [1] Column(width=1000)
        [0] Plotly(Figure)

2023-09-22 03:58:30,301 ERROR: panel.reactive - Callback failed for object named "Participante" changing property {'value': 'JSDR'} 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 384, in _process_events
    self.param.update(**self_events)
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 1902, in update
    self_._batch_call_watchers()
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2063, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2025, in _execute_watcher
    watcher.fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 854, in _replace_pane
    new_object = self.eval(self.object)
  File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 813, in eval
    return eval_function(function)
  File "/usr/local/lib/python3.10

# Original Plot and Base Line

In [6]:
with open('process_data_origin.json', 'r') as json_file:
    origin_data = json.load(json_file)

In [9]:
def plot_2d_eeg(fig, matrix):
  y = np.linspace(0, 700, 175)

  for i in range(len(matrix)):
    fig.add_trace(go.Scatter(
        x = y,
        y = matrix[i],
        marker = dict(size=2),
        line = dict(width=2),
        mode = 'lines',
        name = CHANNELS[i],
        showlegend = True
    ))

  return fig

def make_average(data):
  x = data.sum(axis=0)/34
  return x

def make_base_line(matrix):
  matrix_basel_line = []
  y = np.linspace(0, 700, 175)

  for i in range(len(matrix)):
    baseline_fitter = Baseline(x_data=matrix[i])
    signal_base_line = baseline_fitter.fabc(y)
    matrix_basel_line.append(signal_base_line[0])

  return matrix_basel_line

@pn.depends(participants_selector)
def data_plot(participants_selector):
  fig_6 = go.Figure()
  matrix_avg = make_average(np.asanyarray(origin_data[participants_selector]['6']))
  fig_6 = plot_2d_eeg(fig_6, matrix_avg)

  fig_7 = go.Figure()
  matrix_avg = make_average(np.asanyarray(origin_data[participants_selector]['7']))
  matrix_base_line = make_base_line(matrix_avg)
  fig_7 = plot_2d_eeg(fig_7, matrix_base_line)

  return pn.Row(fig_6, fig_7)

In [17]:
origin_data['AMCJ']['6'][0][1][1]

4.311157032990574e-05

In [10]:
pn.Row(participants_selector, data_plot)


Traceback (most recent call last):
 File "/usr/local/lib/python3.10/dist-packages/pyviz_comms/__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "/usr/local/lib/python3.10/dist-packages/panel/viewable.py", line 472, in _on_msg
 doc.unhold()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/document.py", line 776, in unhold
 self.callbacks.unhold()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 431, in unhold
 self.trigger_on_change(event)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 408, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 443, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/util/callback_manager.py", line 185, in invoke
 callback(attr, old, new)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 469, in _comm_change
 state._handle_exception(e)
 File "/usr/local/lib/python3.10/dist-packages/panel/io/state.py", line 436, in _handle_exception
 raise exception
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 467, in _comm_change
 self._schedule_change(doc, comm)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 449, in _schedule_change
 self._change_event(doc)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 445, in _change_event
 self._process_events(events)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 384, in _process_events
 self.param.update(**self_events)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 1902, in update
 self_._batch_call_watchers()
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2063, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2025, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 854, in _replace_pane
 new_object = self.eval(self.object)
 File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 813, in eval
 return eval_function(function)
 File "/usr/local/lib/python3.10/dist-packages/panel/util/__init__.py", line 325, in eval_function
 return function(*args, **kwargs)
 File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 407, in _depends
 return func(*args, **kw)
 File "<ipython-input-9-5bd61d344502>", line 35, in data_plot
 matrix_avg = make_average(np.asanyarray(origin_data[participants_selector]['6']))
KeyError: 'JSDR'
 


 Traceback (most recent call last):
 File "/usr/local/lib/python3.10/dist-packages/pyviz_comms/__init__.py", line 340, in _handle_msg
 self._on_msg(msg)
 File "/usr/local/lib/python3.10/dist-packages/panel/viewable.py", line 472, in _on_msg
 doc.unhold()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/document.py", line 776, in unhold
 self.callbacks.unhold()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 431, in unhold
 self.trigger_on_change(event)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 408, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "/usr/local/lib/python3.10/dist-packages/bokeh/document/callbacks.py", line 443, in invoke_with_curdoc
 return f()
 File "/usr/local/lib/python3.10/dist-packages/bokeh/util/callback_manager.py", line 185, in invoke
 callback(attr, old, new)
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 469, in _comm_change
 state._handle_exception(e)
 File "/usr/local/lib/python3.10/dist-packages/panel/io/state.py", line 436, in _handle_exception
 raise exception
 File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 467, in _comm_change
 self._schedule_change(doc, comm)
 File "/usr/local/l

Row
    [0] Select(name='Participante', options=['ADJRR', 'CESARMDG', ...], value='CESARMDG')
    [1] ParamFunction(function, _pane=Row, defer_load=False)

2023-09-22 03:57:28,263 ERROR: panel.reactive - Callback failed for object named "Participante" changing property {'value': 'JSDR'} 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/panel/reactive.py", line 384, in _process_events
    self.param.update(**self_events)
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 1902, in update
    self_._batch_call_watchers()
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2063, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "/usr/local/lib/python3.10/dist-packages/param/parameterized.py", line 2025, in _execute_watcher
    watcher.fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 854, in _replace_pane
    new_object = self.eval(self.object)
  File "/usr/local/lib/python3.10/dist-packages/panel/param.py", line 813, in eval
    return eval_function(function)
  File "/usr/local/lib/python3.10